In [2]:
# SET CONFIG AND PARAMETERS
path = '/home/xilinx/jupyter_notebooks/qick/TII/characterizationScripts/zcu216/configuration/'
with open(path+"setConfigV1.py", 'r') as f:
    code1 = f.read()
    exec(code1)
with open(path+"setParametersSpinq10q69.py", 'r') as f:
    code2 = f.read()
    exec(code2)  

Configuration and parameters set


In [8]:
class AmplitudeRabiProgram(NDAveragerProgram):
    
    def ns2cycles(self, ns, gen_ch=None, ro_ch=None):
        return self.us2cycles(ns/1000, gen_ch=gen_ch, ro_ch=ro_ch)

    def initialize(self):
        cfg=self.cfg

        #
        # READOUT
        #
        probe_ch    = cfg["probe_ch"]        
        feedback_ch = cfg["feedback_ch"]        
        ro_pulse_shape  = cfg["ro_pulse_shape"]
        ro_pulse_length = self.ns2cycles(cfg["ro_pulse_length"], gen_ch=probe_ch)  
        self.declare_gen(ch=probe_ch, 
                         nqz=cfg["probe_ch_nqz"], 
                         mux_freqs=self.cfg['ro_pulse_frequency'], 
                         mux_gains=self.cfg['ro_pulse_gain'])
        self.set_pulse_registers(ch=probe_ch, 
                                 style=ro_pulse_shape, 
                                 length=ro_pulse_length, 
                                 mask=[0,1,2,3])        
        for iCh, ch in enumerate(feedback_ch):
            self.declare_readout(ch=ch, 
                                 freq=self.cfg['acquisition_frequency'][iCh], 
                                 length = self.ns2cycles(self.cfg['acquisition_length'], ro_ch=ch), 
                                 gen_ch=probe_ch)  
        sweeps = []
        # Sweeper parameters
        qubit_gain_expts = int((cfg["qubit_gain_stop"] - cfg["qubit_gain_start"])/cfg["qubit_gain_step"])+1
        qubit_gain_start = int(cfg["qubit_gain_start"]*MAX_GAIN)
        qubit_gain_stop = int(cfg["qubit_gain_stop"]*MAX_GAIN) 
        
        for index, qubit in enumerate(cfg["qubits"]):
            drive_ch = cfg["drive_ch"][index]
            drive_pulse_DAC_frequency = self.freq2reg(cfg[qubit]["drive_pulse_frequency"], gen_ch=drive_ch)
            drive_pulse_DAC_phase     = self.deg2reg(cfg[qubit]["drive_pulse_relative_phase"], gen_ch=drive_ch)
            drive_pulse_DAC_gain      = int(cfg[qubit]["drive_pulse_gain"] * MAX_GAIN) #int(cfg["q1_drive_pulse_gain"] * MAX_GAIN)
            drive_pulse_shape         = cfg[qubit]["drive_pulse_shape"]
            drive_pulse_length        = self.ns2cycles(cfg[qubit]["drive_pulse_length"], gen_ch=drive_ch) #self.ns2cycles(cfg["q_length"], gen_ch=q1_drive_ch)            
 
            self.declare_gen(ch=drive_ch, 
                             nqz=cfg["drive_ch_nqz"])
            self.add_gauss(ch=drive_ch, 
                           name="drive_pulse", 
                           sigma=int(cfg[qubit]["sigma"] * 384), 
                           length=drive_pulse_length,)
            self.set_pulse_registers(ch=drive_ch, 
                                     gain=drive_pulse_DAC_gain, 
                                     freq=drive_pulse_DAC_frequency, 
                                     phase=drive_pulse_DAC_phase, 
                                     #style="const",
                                     #length= q1_drive_pulse_length
                                     style=drive_pulse_shape, 
                                     waveform="drive_pulse"
                                    )  
            # ---------- sweep defination starts from here -----------------
            # get freq registers of the generator channel (check 04_Reading_Math_Writing for more details about QickRegister object)
            self.res_r_gain = self.get_gen_reg(drive_ch, "gain") #get register page and gain refor qubit_ch
            # add pulse freq sweep, first added will be first swept
            sweeps.append(QickSweep(self, self.res_r_gain, qubit_gain_start, qubit_gain_stop, qubit_gain_expts))            

        self.add_sweep(merge_sweeps(sweeps))
        
        
        self.synci(500)  # give processor some time to configure pulses
    
    def body(self):
        cfg=self.cfg
        for index, qubit in enumerate(cfg["qubits"]):
            drive_ch = cfg["drive_ch"][index]
            drive_pulse_start = self.ns2cycles(cfg[qubit]["drive_pulse_start"], gen_ch=drive_ch)
            self.pulse(ch=cfg["drive_ch"][index], t=drive_pulse_start)
           

        probe_ch    = cfg["probe_ch"]
        feedback_ch = cfg["feedback_ch"]
        ro_pulse_start       = self.ns2cycles(cfg["qubit_pulse_length"]+cfg["lag"])

        self.measure(pulse_ch= probe_ch, 
                     adcs = feedback_ch, 
                     adc_trig_offset=self.ns2cycles(cfg["delay_before_acquisition"]), 
                     wait=True, 
                     syncdelay=self.ns2cycles(cfg["relaxation_time"]), 
                     t=ro_pulse_start)

In [9]:
experiment_cfg = {  "qubit_gain_start": 0.01,      # Qubit frequency sweeper in MHz
                    "qubit_gain_stop": 1.0,
                    "qubit_gain_step": 0.005,
                    "qubit_pulse_length": 60}        # The same for all the qubits
program_cfg = {**channel_cfg,**readout_cfg,**qubit_cfg, **experiment_cfg} 

In [10]:
def run_rabi_amplitude():
    set_sweetspots()
    
    rabi = AmplitudeRabiProgram(soc_cfg, program_cfg)
    x_pts, avg_di, avg_dq  = rabi.acquire(soc,progress=True)

    reset_bias()

    gain_range = np.transpose(x_pts)  / MAX_GAIN  
    result = np.zeros(np.array(avg_di).shape)
    for i in range(len(avg_di)):
        result[i] = np.abs(avg_di[i] + 1j*avg_dq[i])
    return gain_range, result, rabi

In [11]:

gain_range, pop, program = run_rabi_amplitude()

  0%|          | 0/398000 [00:00<?, ?it/s]

In [17]:
# Create the plot
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=(program_cfg["qubits"][0], program_cfg["qubits"][1], program_cfg["qubits"][2],program_cfg["qubits"][3]))
# Add a scatter plot (line plot)
fig.add_trace(go.Scatter(x=np.squeeze(gain_range[0]), y=np.squeeze(pop[0]), mode='lines+markers', name='Rabi Amplitude Q1'),  row=1, col=1)
fig.add_trace(go.Scatter(x=np.squeeze(gain_range[1]), y=np.squeeze(pop[1]), mode='lines+markers', name='Rabi Amplitude Q2'), row=1, col=2)
fig.add_trace(go.Scatter(x=np.squeeze(gain_range[2]), y=np.squeeze(pop[2]), mode='lines+markers', name='Rabi Amplitude Q3'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.squeeze(gain_range[3]), y=np.squeeze(pop[3]), mode='lines+markers', name='Rabi Amplitude Q4'), row=2, col=2)

# Customize the layout (optional)
fig.update_layout(
    title="Plot of y = sin(x)",
    xaxis_title="X Values",
    yaxis_title="Y Values (sin(x))",
    template="plotly_dark"  # Optional: dark theme
)
    # Update x-axis and y-axis labels
fig.update_yaxes(title_text="Amplitude", tickangle=-45)
fig.update_xaxes(title_text="Gain")
# Customize the layout (optional)
fig.update_layout(
    title={'text': "Rabi Amplitude",
            'font': {
            'size': 24  # Set the font size
        },
          },
    height = 800, width = 1000,
    template="plotly"  # Optional: dark theme
    )
           
# Display the plot
fig.show()

